#### Set systems

In [1]:
%whos

Interactive namespace is empty.


In [2]:
import coderdata as cd
cell_line = cd.DatasetLoader('cell_line', data_directory = '../data/cell_line/') # a
cptac = cd.DatasetLoader('cptac', data_directory = '../data/cptac/') # b
beataml = cd.DatasetLoader('beataml', data_directory = '../data/beataml/') # c
hcmi = cd.DatasetLoader('hcmi', data_directory = '../data/hcmi/') # d

print('DatasetLoaders in RAM')
systems = 'cell-line+CPTAC'

Processing Data...
Processing Data...
Processing Data...
Processing Data...
DatasetLoaders in RAM


In [ ]:
# target output dir: strctrd/one_cncr

#### Semi-interactive devel

In [23]:
import glob
import pandas as pd
import umap
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import glob

##### Functions

In [10]:
# Traspose long-form X-file into biomarkers as columns,
def extract(df_lite):
    import time
    start = time.time()
    data_type = df_lite.columns[2] # extract marker values

    dot_T = df_lite.pivot_table(
        index='improve_sample_id',
        columns='entrez_id',
        values=data_type,
        aggfunc='mean'             # average duplicate values
    )

    end = time.time()
    wall_clock = end - start
    return str(round(wall_clock / 60, 2)) + ' minutes', dot_T

In [11]:
# Extract ids and biomarker values
def df_check(X_n):
    df_lite = X_n.iloc[:, :3] # cut the last two columns, source and study
    size = f"{df_lite.shape[0]:,}"
    na_count = f"{df_lite.isna().sum().sum():,}"
    inf_count = f"{df_lite.isin([np.inf, -np.inf]).sum().sum():,}"
    return df_lite, size, na_count, inf_count

In [12]:
# dot_T = g(d_typ, dot_T.copy())
def g(d_typ, df):
    """
    Checks the data types of columns and index in a DataFrame and prints informative messages.

    Args:
        df (pandas.DataFrame): The DataFrame to check.

    Returns:
        None
    """

    if df.columns.dtype == 'float64' and df.index.dtype == 'float64':
        print('both float')
        df = float_to_string(d_typ, df)
    elif df.columns.dtype == 'float64' and df.index.dtype == 'int':
        print('columns are float, index are int')
        df = indx_int_colm_flt(d_typ, df)
    elif df.columns.dtype == 'int' and df.index.dtype == 'float':
        print('columns are int, index are float, fail, write another function')
        # forth function
    elif df.columns.dtype == 'int' and df.index.dtype == 'int':
        print('columns are int, index are int')
        df = int_to_string(d_typ, df)
    else:
        print('non int or float dtype detected')
    return df

In [13]:
def int_to_string(d_typ, dot_T):
    dot_T.columns = dot_T.columns.map(str)
    dot_T.columns = ['entrz_' + d_typ + i for i in dot_T.columns] #
    dot_T.columns.name = 'entrez_id'

    dot_T.index = dot_T.index.map(str)
    dot_T.index = ['smpl_id_' + i for i in dot_T.index]
    dot_T.index.name = 'improve_sample_id'
    return dot_T

In [14]:
def indx_int_colm_flt(d_typ, dot_T):
    dot_T.columns = dot_T.columns.map(str)
    dot_T.columns = [i.split('.')[0] for i in dot_T.columns]
    dot_T.columns = ['entrz_' + d_typ + i for i in dot_T.columns]
    dot_T.columns.name = 'entrez_id'
    
    dot_T.index = dot_T.index.map(str)
    dot_T.index = ['smpl_id_' + i for i in dot_T.index]
    dot_T.index.name = 'improve_sample_id'
    return dot_T

In [15]:
def float_to_string(d_typ, dot_T):
    dot_T.columns = dot_T.columns.map(str)
    dot_T.columns = [i.split('.')[0] for i in dot_T.columns]
    dot_T.columns = ['entrz_' + d_typ + i for i in dot_T.columns]
    dot_T.columns.name = 'entrez_id'
    
    dot_T.index = dot_T.index.map(str)
    dot_T.index = [i.split('.')[0] for i in dot_T.index]
    dot_T.index = ['smpl_id_' + i for i in dot_T.index]
    dot_T.index.name = 'improve_sample_id'
    return dot_T

##### Systems toggles

In [9]:
systems = 'cell-line+CPTAC'

In [ ]:
systems = 'cell-line+BeatAML'

In [ ]:
systems = 'cell-line+CPTAC'

##### Samples, abstract systems to a and b

In [37]:
# Cell line and CPTAC
sys_a_samp = cell_line.samples
sys_a = 'cell-line'
sys_a_lbl = 'cell_line'

sys_b_samp = cptac.samples
sys_b = 'cptac'
sys_b_lbl = 'CPTAC'

In [35]:
# AML
sys_a_samp = cell_line.samples
sys_a = 'cell-line'
sys_a_lbl = 'cell_line'

sys_b_samp = beataml.samples
sys_b = 'beataml'
sys_b_lbl = 'BeatAML'

##### Cancer_type view

In [19]:
print(sys_a, 'is system a')
sys_a_samp.cancer_type.value_counts()[:60]

cell-line is system a


cancer_type
Lung Adenocarcinoma                            1886
Small Cell Lung Cancer                         1697
Pancreatic Adenocarcinoma                      1657
Melanoma                                       1643
Colon Adenocarcinoma                           1311
Neuroblastoma                                  1054
Glioblastoma                                   1050
Oral Cavity Squamous Cell Carcinoma             938
Diffuse Large B-Cell Lymphoma, NOS              790
Cutaneous Melanoma                              788
Acute Myeloid Leukemia                          747
Ewing Sarcoma                                   722
Invasive Breast Carcinoma                       687
Burkitt Lymphoma                                675
Breast Invasive Ductal Carcinoma                662
T-Lymphoblastic Leukemia/Lymphoma               630
Esophageal Squamous Cell Carcinoma              606
Lung Squamous Cell Carcinoma                    605
Plasma Cell Myeloma                             592


In [20]:
print(sys_b, 'is system b')
sys_b_samp.cancer_type.value_counts()[:50]

cptac is system b


cancer_type
Pancreatic ductal adenocarcinoma         140
Breast carcinoma                         134
Head and Neck squamous cell carcinoma    112
Lung adenocarcinoma                      111
Lung squamous cell carcinoma             110
Ovarian carcinoma                        110
Clear cell renal cell carcinoma          110
Colon adenocarcinoma                     109
Uterine Corpus Endometrial Carcinoma     103
Glioblastoma multiforme                  100
Name: count, dtype: int64

In [ ]:
print(sys_c, 'is system c')
sys_c_samp.cancer_type.value_counts()[:2] # expect one cancer type for beataml

#### Cancer_type toggles, cell line as a, CPTAC as b

In [ ]:
##### single cancers to /strctrd/one_cncr

In [17]:
out_one, cncr, modality, systems

('../strctrd/one_cncr/', '', 'transcriptomics', 'cell-line+CPTAC')

In [22]:
ls ../strctrd/one_cncr/

breast-ductal_transcriptomics_cell-line+CPTAC.tsv
breast-lobular_transcriptomics_cell-line+CPTAC.tsv
breast-nos_transcriptomics_cell-line+CPTAC.tsv


In [28]:
for file in glob.glob('../strctrd/one_cncr/*'):
    one_chk = pd.read_csv(file,
                    sep = '\t', index_col = 0)

    break

In [ ]:
one_chk

In [ ]:
# Lung adeno
cncr = 'lung-adeno'
cncr_lbl = 'lung_adeno'
sys_a_samp_canc_n = sys_a_samp[sys_a_samp.cancer_type == 'Lung Adenocarcinoma'] # cl
sys_b_samp_canc_n = sys_b_samp[sys_b_samp.cancer_type == 'Lung adenocarcinoma'] # cp

In [ ]:
# Pancreatic adeno
cncr = 'pancreatic-adeno'
cncr_lbl = 'pancreatic_adeno'
sys_a_samp_canc_n = sys_a_samp[sys_a_samp.cancer_type == 'Pancreatic Adenocarcinoma'] # cl
sys_b_samp_canc_n = sys_b_samp[sys_b_samp.cancer_type == 'Pancreatic ductal adenocarcinoma'] # cp

In [ ]:
# Head and neck
cncr = 'head-neck'
cncr_lbl = 'head_neck'
sys_a_samp_canc_n = sys_a_samp[sys_a_samp.cancer_type == 'Head and Neck Squamous Cell Carcinoma'] # cl
sys_b_samp_canc_n = sys_b_samp[sys_b_samp.cancer_type == 'Head and Neck squamous cell carcinoma'] # cp

In [30]:
# build non-brca cell line to cptac for production run
names = ['lung-adeno', 'pancreatic-adeno', 'head-neck', 'colon-adeno', 'glioblastoma', 'renal-clear-cell']
labels = ['lung_adeno', 'pancreatic_adeno', 'head_neck', 'colon_adeno', 'glioblastoma', 'renal-clear_cell']
a_list = ['Lung Adenocarcinoma',
          'Pancreatic Adenocarcinoma',
          'Head and Neck Squamous Cell Carcinoma',
          'Colon Adenocarcinoma',
          'Glioblastoma',
          'Renal Clear Cell Carcinoma']
b_list = ['Lung adenocarcinoma',
          'Pancreatic ductal adenocarcinoma',
          'Head and Neck squamous cell carcinoma',
          'Colon adenocarcinoma',
          'Glioblastoma multiforme',
          'Clear cell renal cell carcinoma']

In [ ]:
# Colon adeno
cncr = 'colon-adeno'
cncr_lbl = 'colon_adeno'
sys_a_samp_canc_n = sys_a_samp[sys_a_samp.cancer_type == 'Colon Adenocarcinoma'] # cl
sys_b_samp_canc_n = sys_b_samp[sys_b_samp.cancer_type == 'Colon adenocarcinoma'] # cp

In [104]:
# Glioblastoma
cncr = 'glioblastoma'
cncr_lbl = 'glioblastoma'
sys_a_samp_canc_n = sys_a_samp[sys_a_samp.cancer_type == 'Glioblastoma']
sys_b_samp_canc_n = sys_b_samp[sys_b_samp.cancer_type == 'Glioblastoma multiforme'] # cp

In [139]:
# Renal clear cell
cncr = 'renal-clear-cell'
cncr_lbl = 'renal_clear_cell'
sys_a_samp_canc_n = sys_a_samp[sys_a_samp.cancer_type == 'Renal Clear Cell Carcinoma']
sys_b_samp_canc_n = sys_b_samp[sys_b_samp.cancer_type == 'Clear cell renal cell carcinoma'] # cp

#### BRCA

In [22]:
# Breast Ductal
cncr = 'breast-ductal'
cncr_lbl = 'breast_ductal'
sys_a_samp_canc_n = sys_a_samp[sys_a_samp.cancer_type == 'Breast Invasive Ductal Carcinoma'] # cl
sys_b_samp_canc_n = sys_b_samp[sys_b_samp.cancer_type == 'Breast carcinoma']

In [ ]:
# Breast Lobular
cncr = 'breast-lobular'
cncr_lbl = 'breast_lobular'
sys_a_samp_canc_n = sys_a_samp[sys_a_samp.cancer_type == 'Breast Invasive Lobular Carcinoma'] # cl
sys_b_samp_canc_n = sys_b_samp[sys_b_samp.cancer_type == 'Breast carcinoma']

In [ ]:
# Breast NOS
cncr = 'breast-nos'
cncr_lbl = 'breast_nos'
sys_a_samp_canc_n = sys_a_samp[sys_a_samp.cancer_type == 'Breast Invasive Carcinoma, NOS'] # cl
sys_b_samp_canc_n = sys_b_samp[sys_b_samp.cancer_type == 'Breast carcinoma']

##### brca single cancers to /strctrd/one_cncr

In [31]:
pwd

'/Users/karlberb/Documents/work/2024/bnch_mrk/mdl-sys-bnchmrk/code'

In [43]:
ls

create_lsp_v1.sh*       mbatch/                 schedule_vae_cpu.sh*
default_vae.py          mbatch_i-o.ipynb        schedule_vae_gpu.sh*
dstnc.ipynb             mmd.py                  strctr.py
dta_ldr.ipynb           nb_dvl/                 vae.py*
frnt_end_clf_ump.ipynb  plots/
job_scheduler.sh*       schedule_lsp_v1.sh*


In [31]:
# strctr_one.py
# Script to build one_cncr files
    # Sript for building strctrd/two_cncr files
    # will read from strctrd/one_cncr files
# pwd: /mdl-sys-bnchmrk/code

# Imports <-------------------------------- Devel toggle
# import coderdata as cd
# import glob
# import pandas as pd
# import umap
# import numpy as np
# import os
# import matplotlib.pyplot as plt
# from matplotlib.lines import Line2D

# pwd
# mdl-sys-bnchmrk/code

# ls
# strctr.py

# Import functions
import strctr

systems = 'cell-line+CPTAC'

# Read data (construct loader objects) <--- Devel toggle
# cell_line = cd.DatasetLoader(
#   'cell_line', data_directory = '../data/cell_line/') # a 
# cptac = cd.DatasetLoader(
#   'cptac', data_directory = '../data/cptac/') # b
print('Data loader modules built')

# Cell lines are system A
sys_a_samp = cell_line.samples
sys_a = 'cell-line'
sys_a_lbl = 'cell_line'

# CPTAC is system B
sys_b_samp = cptac.samples
sys_b = 'cptac'
sys_b_lbl = 'CPTAC'

# BeatAML is system B
# sys_b_samp = beataml.samples
# sys_b = 'beat-aml'
# sys_b_lbl = 'BeatAML'

# Transcriptomics data modality extraction
modality = 'transcriptomics' # to file name
moda = 'tran_' # to columns and index

# Proteomics data modality extraction
# modality = 'proteomics' # to file name
# moda = 'prot_' # to columns and index

# Copy number and Mutations
# Data modality extractions insertion point

# BRCA discrete labels <---------------------------
# names = ['breast-ductal', 'breast-lobular', 'breast-nos']
# labels = ['breast_ductal', 'breast_lobular', 'breast_nos']
# a_list = ['Breast Invasive Ductal Carcinoma',
#           'Breast Invasive Lobular Carcinoma',
#           'Breast Invasive Carcinoma, NOS']
# b_list = ['Breast carcinoma',
#           'Breast carcinoma',
#           'Breast carcinoma']

# BRCA cell line unified labels <------------------
# names = ['breast-ductal', 'breast-lobular', 'breast-nos']
# labels = ['breast_ductal', 'breast_lobular', 'breast_nos']
# a_list = ['Breast Invasive Ductal Carcinoma']
# b_list = ['Breast carcinoma']

# BeatAML direct mapping to cell line <------------
# names = ['']
# labels = ['']
# a_list = ['']
# b_list = ['']

# Cell line and CPTAC production run <------------
names = ['lung-adeno',
         'pancreatic-adeno',
         'head-neck',
         'colon-adeno',
         'glioblastoma',
         'renal-clear-cell']
labels = ['lung_adeno',
          'pancreatic_adeno',
          'head_neck',
          'colon_adeno',
          'glioblastoma',
          'renal-clear_cell']
a_list = ['Lung Adenocarcinoma',
          'Pancreatic Adenocarcinoma',
          'Head and Neck Squamous Cell Carcinoma',
          'Colon Adenocarcinoma',
          'Glioblastoma',
          'Renal Clear Cell Carcinoma']
b_list = ['Lung adenocarcinoma',
          'Pancreatic ductal adenocarcinoma',
          'Head and Neck squamous cell carcinoma',
          'Colon adenocarcinoma',
          'Glioblastoma multiforme',
          'Clear cell renal cell carcinoma']

for i, cncr in enumerate(names):
    # cncr = a_list[i]
    cncr_lbl = labels[i]
    
    sys_a_samp_canc_n = sys_a_samp[sys_a_samp.cancer_type == a_list[i]]
    sys_b_samp_canc_n = sys_b_samp[sys_b_samp.cancer_type == b_list[i]]

    ids_sys_a = sys_a_samp_canc_n.improve_sample_id # cl
    ids_sys_b = sys_b_samp_canc_n.improve_sample_id # cp

    mda_n_sys_a = cell_line.transcriptomics[cell_line.transcriptomics.improve_sample_id.isin(ids_sys_a)] # cl
    mda_n_sys_b = cptac.transcriptomics[cptac.transcriptomics.improve_sample_id.isin(ids_sys_b)]
    # break
    df_lite, size, na_count, inf_count = df_check(mda_n_sys_a)
    # df_lite, size, na_count, inf_count = strctr.df_check(mda_n_sys_a)
    print(sys_a, '| sys a')
    print(cncr, modality)
    print('len: ', size)
    print('NaNs: ', na_count)
    print('Infs: ', inf_count)

    wall_clock, dot_T = extract(df_lite)
    dot_T = g(moda, dot_T.copy())
    dot_T.dropna(axis = 1, inplace = True)
    a = dot_T # cell line

    df_lite, size, na_count, inf_count = df_check(mda_n_sys_b)
    # df_lite, size, na_count, inf_count = strctr.df_check(mda_n_sys_a)
    print(sys_b, '| sys b')
    print(cncr, modality)
    print('len: ', size)
    print('NaNs: ', na_count)
    print('Infs: ', inf_count)
    
    wall_clock, dot_T = extract(df_lite)
    dot_T = g(moda, dot_T.copy())
    dot_T.dropna(axis = 1, inplace = True)
    b = dot_T # cptac
    
    a.insert(0, 'Cancer_type', cncr_lbl)
    b.insert(0, 'Cancer_type', cncr_lbl)
    a.insert(0, 'System', sys_a_lbl)
    b.insert(0, 'System', sys_b_lbl)

    ab = pd.concat([a, b], axis=0, join='inner')
    print(ab.System.value_counts())
    print(ab.Cancer_type.value_counts())

    out_one = '../strctrd/one_cncr/'
    print('disk break')
    # break
    # Write two-system, single cancer type to disk <--- Devel toggle 
    ab.to_csv(
        '../strctrd/'+out_one+'/'+cncr+'_'+modality+'_'+systems+'.tsv',
        sep = '\t')
    # break
print('One-cancer production run complete')

Data loader modules built
cell-line | sys a
lung-adeno transcriptomics
len:  1,477,802
NaNs:  0
Infs:  0
columns are int, index are int
cptac | sys b
lung-adeno transcriptomics
len:  4,420,275
NaNs:  115,532
Infs:  0
both float
System
CPTAC        111
cell_line     77
Name: count, dtype: int64
Cancer_type
lung_adeno    188
Name: count, dtype: int64
disk break
cell-line | sys a
pancreatic-adeno transcriptomics
len:  1,055,572
NaNs:  0
Infs:  0
columns are int, index are int
cptac | sys b
pancreatic-adeno transcriptomics
len:  5,572,972
NaNs:  142,658
Infs:  0
both float
System
CPTAC        140
cell_line     55
Name: count, dtype: int64
Cancer_type
pancreatic_adeno    195
Name: count, dtype: int64
disk break
cell-line | sys a
head-neck transcriptomics
len:  95,954
NaNs:  0
Infs:  0
columns are int, index are int
cptac | sys b
head-neck transcriptomics
len:  4,340,368
NaNs:  111,792
Infs:  0
both float
System
CPTAC        109
cell_line      5
Name: count, dtype: int64
Cancer_type
head_nec

In [ ]:
# disk write check

In [32]:
ls ../strctrd/one_cncr/

breast-ductal_transcriptomics_cell-line+CPTAC.tsv
breast-lobular_transcriptomics_cell-line+CPTAC.tsv
breast-nos_transcriptomics_cell-line+CPTAC.tsv
colon-adeno_transcriptomics_cell-line+CPTAC.tsv
glioblastoma_transcriptomics_cell-line+CPTAC.tsv
head-neck_transcriptomics_cell-line+CPTAC.tsv
lung-adeno_transcriptomics_cell-line+CPTAC.tsv
pancreatic-adeno_transcriptomics_cell-line+CPTAC.tsv
renal-clear-cell_transcriptomics_cell-line+CPTAC.tsv


In [33]:
for file in glob.glob('../strctrd/one_cncr/*'):
    one_chk = pd.read_csv(file,
                    sep = '\t', index_col = 0)

    break

In [ ]:
one_chk

##### Merged BRCA as single cancer

##### intertools, brca, two cancer

In [21]:
from itertools import combinations

#### Cancer_type toggles, cell line as a, BeatAML as c

In [ ]:
# devel, c = b
cncr = 'beat-aml'
cncr_lbl = 'beat_aml'
sys_a_samp_canc_n = sys_a_samp[sys_a_samp.cancer_type == 'Renal Clear Cell Carcinoma']
sys_c_samp_canc_n = sys_c_samp[sys_c_samp.cancer_type == 'Clear cell renal cell carcinoma']

In [ ]:
# beat aml as single cancer


#### Extract sample ids

In [26]:
print(cncr)
ids_sys_a = sys_a_samp_canc_n.improve_sample_id # cl
ids_sys_b = sys_b_samp_canc_n.improve_sample_id # cp

breast-ductal


In [ ]:
print(cncr)
ids_sys_a = sys_a_samp_canc_n.improve_sample_id # cl
ids_sys_b = sys_b_samp_canc_n.improve_sample_id # beataml

#### Data extraction

In [ ]:
modality = 'transcriptomics' # to file name
moda = 'tran_' # to columns and index
mda_n_sys_a = cell_line.transcriptomics[cell_line.transcriptomics.improve_sample_id.isin(ids_sys_a)] # cl
mda_n_sys_b = cptac.transcriptomics[cptac.transcriptomics.improve_sample_id.isin(ids_sys_b)]

In [ ]:
modality = 'proteomics' # to file name
moda = 'prot_' # to columns and index
mda_n_sys_a = cell_line.proteomics[cell_line.proteomics.improve_sample_id.isin(ids_proj_a)] # cl
mda_n_sys_b = cptac.proteomics[cptac.proteomics.improve_sample_id.isin(ids_proj_b)]

In [ ]:
modality = 'copy-number' # to file name
moda = 'cpnm_' # to columns and index
mda_n_sys_a= cell_line.copy_number[cell_line.copy_number.improve_sample_id.isin(ids_proj_a)]
mda_n_sys_b= cell_line.copy_number[cell_line.copy_number.improve_sample_id.isin(ids_proj_b)]

In [ ]:
modality = 'mutations' # to file name
moda = 'muta_' # to columns and index
mda_n_sys_a = cptac.mutations[cptac.mutations.improve_sample_id.isin(ids_proj_a)]
mda_n_sys_b = cell_line.mutations[cell_line.mutations.improve_sample_id.isin(ids_proj_b)]

#### Project A, cell line

In [47]:
df_lite, size, na_count, inf_count = df_check(mda_n_sys_a)
print(sys_a, '| sys a')
print(cncr, modality)
print(size)
print(na_count)
print(inf_count)

wall_clock, dot_T = extract(df_lite)
dot_T = g(moda, dot_T.copy())
dot_T.dropna(axis = 1, inplace = True)
a = dot_T # cell line

cell-line | sys a
breast-ductal transcriptomics
556,574
0
0
columns are int, index are int


##### a devel

In [143]:
wall_clock, dot_T = extract(df_lite)
print(wall_clock)
dot_T.iloc[:3, :3]

0.0 minutes


entrez_id,1,2,9
improve_sample_id,,,
15,0.176323,0.097611,1.778209
158,0.201634,0.669027,2.127633
249,0.111031,0.214125,2.140779


In [144]:
cncr

'renal-clear-cell'

In [45]:
moda

'tran_'

In [146]:
# dot_T = g(d_typ, dot_T.copy())
dot_T = g(moda, dot_T.copy())
dot_T.iloc[:3, :3]

columns are int, index are int


entrez_id,entrz_tran_1,entrz_tran_2,entrz_tran_9
improve_sample_id,,,
smpl_id_15,0.176323,0.097611,1.778209
smpl_id_158,0.201634,0.669027,2.127633
smpl_id_249,0.111031,0.214125,2.140779


In [147]:
dot_T.shape

(17, 19176)

In [148]:
dot_T.dropna(axis = 1, inplace = True)

In [149]:
dot_T.shape

(17, 19176)

In [150]:
a = dot_T # cell line

#### Project B, CPTAC

In [48]:
df_lite, size, na_count, inf_count = df_check(mda_n_sys_b)
print(sys_b, '| sys b')
print(cncr, modality)
print('len: ', size)
print('NaNs: ', na_count)
print('Infs: ', inf_count)

wall_clock, dot_T = extract(df_lite)
dot_T = g(moda, dot_T.copy())
dot_T.dropna(axis = 1, inplace = True)
b = dot_T # cptac

cptac | sys b
breast-ductal transcriptomics
len:  5,027,572
NaNs:  96,937
Infs:  0
both float


##### b devel

In [152]:
wall_clock, dot_T = extract(df_lite)
print(wall_clock)
dot_T.iloc[:3, :3]

0.02 minutes


entrez_id,1.0,2.0,3.0
improve_sample_id,,,
2891.0,5.33,15.85,5.32
2892.0,4.96,15.87,4.98
2893.0,4.84,15.52,4.50


In [153]:
dot_T = g(moda, dot_T.copy())

both float


In [154]:
dot_T.iloc[:3, :3]

entrez_id,entrz_tran_1,entrz_tran_2,entrz_tran_3
improve_sample_id,,,
smpl_id_2891,5.33,15.85,5.32
smpl_id_2892,4.96,15.87,4.98
smpl_id_2893,4.84,15.52,4.50


In [155]:
dot_T.shape

(110, 37287)

In [156]:
dot_T.dropna(axis = 1, inplace = True)

In [157]:
dot_T.shape

(110, 37287)

In [158]:
b = dot_T # cptac

#### Labels

##### Check a and b are different

In [160]:
a.iloc[:3, :3]

entrez_id,entrz_tran_1,entrz_tran_2,entrz_tran_9
improve_sample_id,,,
smpl_id_15,0.176323,0.097611,1.778209
smpl_id_158,0.201634,0.669027,2.127633
smpl_id_249,0.111031,0.214125,2.140779


In [161]:
b.iloc[:3, :3]

entrez_id,entrz_tran_1,entrz_tran_2,entrz_tran_3
improve_sample_id,,,
smpl_id_2891,5.33,15.85,5.32
smpl_id_2892,4.96,15.87,4.98
smpl_id_2893,4.84,15.52,4.50


##### Continue

In [162]:
a.insert(0, 'Cancer_type', cncr_lbl)
b.insert(0, 'Cancer_type', cncr_lbl)
a.insert(0, 'System', sys_a_lbl)
b.insert(0, 'System', sys_b_lbl)

In [163]:
ab = pd.concat([a, b], axis=0, join='inner')

In [ ]:
ab

In [165]:
ab.System.value_counts()

System
CPTAC        110
cell_line     17
Name: count, dtype: int64

In [166]:
ab.Cancer_type.value_counts()

Cancer_type
renal_clear_cell    127
Name: count, dtype: int64

In [167]:
# Set target output dirs
out_dbl = '../output/dbl-cncr/'
out_sng = '../output/sng-cncr'
ump_out = '../output/umap/'

In [168]:
# out, u_o, cancer, data_type
out_dbl, out_sng, cncr, modality, ump_out

('../output/dbl-cncr/',
 '../output/sng-cncr',
 'renal-clear-cell',
 'transcriptomics',
 '../output/umap/')

In [169]:
# Write two-system, single cancer type to disk
ab.to_csv(
    '../output/'+out_sng+'/'+cncr+'_'+modality+'_'+systems+'.tsv',
    sep = '\t')

In [170]:
sng_chk = pd.read_csv('../output/'+out_sng+'/'+cncr+'_'+modality+'_'+systems+'.tsv',
                    sep = '\t', index_col = 0)

In [ ]:
print(cncr)
sng_chk

#### Dbl Cncr

In [178]:
print(cncr)
ab.iloc[:3, :3]

renal-clear-cell


entrez_id,System,Cancer_type,entrz_tran_1
improve_sample_id,,,
smpl_id_15,cell_line,renal_clear_cell,0.176323
smpl_id_158,cell_line,renal_clear_cell,0.201634
smpl_id_249,cell_line,renal_clear_cell,0.111031


In [179]:
ab.shape

(127, 19133)

#### Disk read Cancer_n

In [ ]:
# Interactive zone for batch correction experiments

In [ ]:
# Template read-in path
output/sng_cncr/lung_adeno_transcriptomics_cell_line_CPTAC.tsv

#### Experimental record

In [ ]:
# toggle zone 1

In [198]:
systems = 'cell-line+CPTAC'

In [215]:
cncr_n = 'renal-clear-cell'
moda_n = 'transcriptomics'
sys_n = 'cell-line+CPTAC'

In [ ]:
cncr_n = 'pancreatic-adeno'
moda_n = 'transcriptomics'
sys_n = 'cell-line+CPTAC'

In [220]:
cncr_n = 'lung-adeno'
moda_n = 'transcriptomics'
sys_n = 'cell-line+CPTAC'

In [233]:
cncr_n = 'glioblastoma'
moda_n = 'transcriptomics'
sys_n = 'cell-line+CPTAC'

In [247]:
cncr_n = 'colon-adeno'
moda_n = 'transcriptomics'
sys_n = 'cell-line+CPTAC'

In [ ]:
# toggle zone 2

In [218]:
c1 = pd.read_csv( # single cancer 1
    '../output/sng-cncr/'+cncr_n+'_'+moda_n+'_'+sys_n+'.tsv',
    sep = '\t', index_col = 0)

In [219]:
c1.iloc[:3, :3]

,System,Cancer_type,entrz_tran_1
improve_sample_id,,,
smpl_id_15,cell_line,renal_clear_cell,0.176323
smpl_id_158,cell_line,renal_clear_cell,0.201634
smpl_id_249,cell_line,renal_clear_cell,0.111031


In [248]:
c2 = pd.read_csv( # single cancer 1
    '../output/sng-cncr/'+cncr_n+'_'+moda_n+'_'+sys_n+'.tsv',
    sep = '\t', index_col = 0)

In [249]:
c2.iloc[:3, :3]

,System,Cancer_type,entrz_tran_1
improve_sample_id,,,
smpl_id_3,cell_line,colon_adeno,1.400538
smpl_id_7,cell_line,colon_adeno,0.137504
smpl_id_9,cell_line,colon_adeno,1.464668


In [250]:
print(moda_n)
dbl_cncr = pd.concat([c1, c2], axis = 0, join = 'inner')
dbl_cncr.shape

transcriptomics


(292, 19133)

In [251]:
dbl_cncr.iloc[:3, :3]

,System,Cancer_type,entrz_tran_1
improve_sample_id,,,
smpl_id_15,cell_line,renal_clear_cell,0.176323
smpl_id_158,cell_line,renal_clear_cell,0.201634
smpl_id_249,cell_line,renal_clear_cell,0.111031


In [252]:
dbl_cncr.iloc[-3:, :3]

,System,Cancer_type,entrz_tran_1
improve_sample_id,,,
smpl_id_2102,CPTAC,colon_adeno,2.00
smpl_id_2103,CPTAC,colon_adeno,0.00
smpl_id_2104,CPTAC,colon_adeno,1.48


In [253]:
# two sets of number for each label type

In [254]:
dbl_cncr.System.value_counts()

System
CPTAC        215
cell_line     77
Name: count, dtype: int64

In [255]:
dbl_cncr.Cancer_type.value_counts()

Cancer_type
colon_adeno         165
renal_clear_cell    127
Name: count, dtype: int64

In [256]:
out_dbl, cncr, cncr_n, moda_n, systems

('../output/dbl-cncr/',
 'renal-clear-cell',
 'colon-adeno',
 'transcriptomics',
 'cell-line+CPTAC')

In [257]:
ls ../output/dbl-cncr/

2024-02-17/
pancreatic-adeno+lung-adeno_transcriptomics_cell-line+CPTAC.tsv
renal-clear-cell+glioblastoma_transcriptomics_cell-line+CPTAC.tsv
renal-clear-cell+lung-adeno_transcriptomics_cell-line+CPTAC.tsv
renal-clear-cell+pancreatic-adeno_transcriptomics_cell-line+CPTAC.tsv


In [258]:
dbl_cncr.to_csv('../output/'+out_dbl+'/'+cncr+'+'+cncr_n+'_'+moda_n+'_'+systems+'.tsv',
                    sep = '\t')

In [259]:
dbl_chk = pd.read_csv('../output/'+out_dbl+'/'+cncr+'+'+cncr_n+'_'+moda_n+'_'+systems+'.tsv',
                    sep = '\t', index_col = 0)

In [ ]:
dbl_chk